In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

from Baseline import Baseline
from RegressionTrainingTools import Trainer
device = torch.device('cuda')

In [2]:
x_train = pd.read_csv('data/train_preprocessed.csv', engine='pyarrow')
x_test = pd.read_csv('data/test_preprocessed.csv', engine='pyarrow')

In [3]:
x_train.head()

,ari_co,ari_po,ship_type_category,dist,breadth,built,depth,draught,gt,u_wind,...,air_temperature,bn,ata_lt,dubai,bdi_adj,port_size,ci_hour,month,wind_speed,deadweight_group
0,0,0,0,32.590869,40.0,28,20.0,20.0,86100,-0.256667,...,17.050794,4.127843,21,98.07,1152.458360,0.000113,161.218056,9,1.591468,1
1,0,0,0,35.575496,30.0,20,20.0,10.0,29400,-0.256667,...,17.050794,4.127843,11,99.03,1141.586111,0.000113,95.767500,9,1.591468,1
2,0,0,0,40.909139,40.0,13,20.0,10.0,48200,-0.256667,...,17.050794,4.127843,11,100.39,1135.655794,0.000113,35.445556,9,1.591468,1
3,0,0,0,45.939559,40.0,11,20.0,10.0,58600,-0.256667,...,17.050794,4.127843,11,99.03,1141.586111,0.000113,95.507222,9,1.591468,1
4,0,0,0,15.606497,30.0,11,20.0,10.0,44300,-0.256667,...,17.050794,4.127843,11,99.03,1141.586111,0.000113,99.873056,9,1.591468,1


# Preprocessing

In [4]:
from sklearn.preprocessing import MinMaxScaler
scaling_columns = ['dist', 'breadth', 'built', 'depth', 'draught', 'gt', 'air_temperature', 'bn', 'dubai', 'bdi_adj', 'port_size', 'wind_speed']
scaler = MinMaxScaler(feature_range=(-1, 1))
x_train.loc[:, scaling_columns] = scaler.fit_transform(x_train.loc[:, scaling_columns])
x_test.loc[:, scaling_columns] = scaler.transform(x_test.loc[:, scaling_columns])

In [5]:
x_train.head()

,ari_co,ari_po,ship_type_category,dist,breadth,built,depth,draught,gt,u_wind,...,air_temperature,bn,ata_lt,dubai,bdi_adj,port_size,ci_hour,month,wind_speed,deadweight_group
0,0,0,0,-0.674062,0.2,-0.211268,0.333333,1.0,-0.274224,-0.256667,...,0.234356,-0.261544,21,0.478923,-0.616037,-0.916749,161.218056,9,-0.898146,1
1,0,0,0,-0.644213,-0.2,-0.436620,0.333333,0.0,-0.753008,-0.256667,...,0.234356,-0.261544,11,0.495715,-0.620860,-0.916749,95.767500,9,-0.898146,1
2,0,0,0,-0.590871,0.2,-0.633803,0.333333,0.0,-0.594258,-0.256667,...,0.234356,-0.261544,11,0.519503,-0.623490,-0.916749,35.445556,9,-0.898146,1
3,0,0,0,-0.540562,0.2,-0.690141,0.333333,0.0,-0.506439,-0.256667,...,0.234356,-0.261544,11,0.495715,-0.620860,-0.916749,95.507222,9,-0.898146,1
4,0,0,0,-0.843922,-0.2,-0.690141,0.333333,0.0,-0.627190,-0.256667,...,0.234356,-0.261544,11,0.495715,-0.620860,-0.916749,99.873056,9,-0.898146,1


In [6]:
print(x_train.shape[1]-1 == x_test.shape[1])

True


# Save

In [7]:
x_test.to_csv('data/test_4dl.csv', encoding='UTF-8', index=False)

# Data Preparation

In [8]:
y_train = np.sqrt(x_train.ci_hour.copy())
x_train.drop(columns=['ci_hour'], inplace=True)

x_train = torch.FloatTensor(x_train.values)
y_train = torch.FloatTensor(y_train.values)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, random_state=42, test_size=0.2)

train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False)

# Modeling

In [9]:
model = Baseline(input_dim=20).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

In [10]:
trainer = Trainer(criterion, device, save_path='checkpoints/best_model.pt')
best_model = trainer.train(model, optimizer, train_loader, val_loader, patience=7, epochs=100)

Epoch 55: 100%|██████████| 1375/1375 [00:04<00:00, 291.07it/s, Train Loss=32.3737, Valid Loss=32.4802, Valid R2=0.2163, Valid MAE=3.7877]

Early Stopped


In [11]:
trainer.test(val_loader)

Test Loss: 32.4588 | Test R2: 0.2169 | Test MAE: 3.8401
